# Review Classification
#### In this project I am going to classify reviews in to positive and negetive reviews. I am going to use Embedding, LSTM and a fully connected layer at the end. For sake of learning I am not going to use any pretrained model. 

##### I am running on RTX 4070 gpu. At first I check how many unique words are in my dataset so I choose between word level and char level. I could use pretrained ones but I want to do it all by myself.  

### Status: In progress